<a href="https://colab.research.google.com/github/yueguo1997/Group4_Low_Demensional_Representation_with_Big_Genomics_Data/blob/main/Bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Package

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
cur_path = "/content/drive/MyDrive/Bigdataproject/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Bigdataproject


In [3]:
!pip install pyspark
!pip install pyspark[sql]
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
import pyspark as spark
import time
from operator import add
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

In [23]:
import numpy as np
import random
import pandas as pd

# Read data

In [6]:
# set spark session
ss = (SparkSession
  .builder
  .master("local[5]")
  .appName("fqproject")
  .getOrCreate())

In [7]:
# set spark session
sc = (spark
  .SparkContext
  .getOrCreate(spark
    .SparkConf()
    .setAppName("fqproject")
    .setMaster('local[*]')
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '4G')
    .set('spark.driver.maxResultSize', '4G')))

In [8]:
# read reads.fq and extract the second line of every four lines.
reads = (sc
  .textFile(cur_path + "data/reads.fq")
  .zipWithIndex()
  .filter(lambda x: (x[1]+1)%2==0 and (x[1]+1)%4!=0)
  .map(lambda x: x[0]))

In [9]:
# read reference
reference = (ss
  .read
  .csv(
      cur_path + "data/reference_chr21_20000000_20050000.fa",
      inferSchema=True,
      header=True))

# kmer function

In [10]:
def extract_kmers(s, k):
    return [s[i:i+k] for i in range(len(s)-k+1)]

In [11]:
def check_kmer_features_old(read,name_list):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (name_list[read],Vectors.sparse(len(kmers_list),m,v))

# feature function

In [12]:
r_list = reads.collect()
read_name = {r_list[i]: 'reads' + str(i) for i in range(len(r_list))}


In [13]:
reference_rdd = reference.select("sequence").rdd.map(lambda x: x['sequence'])
reference_list = reference_rdd.collect()
start_rdd = reference.select("start").rdd.map(lambda x: x['start'])
end_rdd = reference.select("end").rdd.map(lambda x: x['end'])
s_list = start_rdd.collect()
e_list = end_rdd.collect()
reference_name = ['bin' + str(s_list[i]) + '_' +  str(e_list[i]) for i in range(len(s_list))]
reference_name_d = {reference_list[i]: reference_name[i] for i in range(len(reference_list))}

# MinHash

In [14]:
# define minhash
def minhash(hashtable: list, entry: tuple) -> tuple:
  ans = []
  for ls in hashtable:
    for i in ls:
      if entry[1][ls[i]]==1:
        ans.append(i)
        break
  return (entry[0], ans)

In [15]:
def minhash_table(n: int) -> list:
  ans = []
  for i in range(n):
    ans.append(random.sample(range(len(kmers)), len(kmers)))
  return ans

In [16]:
random.seed(114514)

# generate hashes and write

In [30]:
#define the range
kmer_range = [5,10,15,20]
permunate = [500,1000,2000]

In [ ]:
# loop every settings
for k_n in kmer_range:
  for p_n in permunate:
    # generate kmers
    reads_kmers = reads.map(lambda x: extract_kmers(x,k_n))
    reference_kmers = (reference.select('sequence').rdd.flatMap(lambda x: x).map(lambda x: extract_kmers(x,k_n)))
    kmers = set(reads_kmers.union(reference_kmers).flatMap(lambda x: x) .distinct().collect())

    # use kemers to check the read and reference features
    reads_features = reads.map(lambda x: check_kmer_features_old(x,read_name))
    reference_features = reference_rdd.map(lambda x: check_kmer_features_old(x,reference_name_d))

    print(k_n,p_n,"features generation finished")

    # generate hash table
    hashtable = minhash_table(p_n)
    print(k_n,p_n,"hash table finished")

    #generate read hash features
    read_sig = reads_features.map(lambda x: minhash(hashtable, x))
    start = time.time()
    print(time.localtime(start))
    read_sig = reads_features.map(lambda x: minhash(hashtable, x))
    print('read hash generation time', time.time()-start)

    #generate ref hash features
    start = time.time()
    print(time.localtime(start))
    ref_sig = reference_features.map(lambda x: minhash(hashtable, x))
    print('ref hash generation time', time.time()-start)
    
    read_l = read_sig.collect()
    ref_l = ref_sig.collect()

    print('hash list generated')

    # generate similarities

    final_ref = []
    final_score = []
    for i in read_l:
      max_score = 0
      max_ref = ''
      for j in ref_l:
        score = len([1 for e in range(p_n) if i[1][e] == j[1][e]])/p_n
        if score > max_score:
          max_ref = j[0]
          max_score = score
      final_ref.append(max_ref)
      final_score.append(max_score)
    final_read = [i[0] for i in read_l]
    print(k_n,p_n,"similarity finished")
    final_df = pd.DataFrame({'read': final_read,'reference': final_ref,'score': final_score})
    file_name = 'final_result_k' + str(k_n) + '_p' + str(p_n) + '.csv'
    final_df.to_csv(cur_path + "data/" + file_name, header=True, index=None, sep=',')
    print(k_n,p_n,"csv file finished")
      

    print('-----------------------------------------------------------')






5 500 features generation finished
5 500 hash table finished
time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=23, tm_min=51, tm_sec=8, tm_wday=1, tm_yday=101, tm_isdst=0)
read hash generation time 5.6743621826171875e-05
time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=23, tm_min=51, tm_sec=8, tm_wday=1, tm_yday=101, tm_isdst=0)
ref hash generation time 8.320808410644531e-05
hash list generated
5 500 similarity finished
5 500 csv file finished
-----------------------------------------------------------
5 1000 features generation finished
5 1000 hash table finished
time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=23, tm_min=54, tm_sec=32, tm_wday=1, tm_yday=101, tm_isdst=0)
read hash generation time 7.82012939453125e-05
time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=23, tm_min=54, tm_sec=32, tm_wday=1, tm_yday=101, tm_isdst=0)
ref hash generation time 3.3855438232421875e-05
hash list generated
5 1000 similarity finished
5 1000 csv file fi

In [ ]:
# because of the long runing time. I split this operation in two times. The notebook may not display all the result